In [1]:
import pandas as pd
import numpy as np
import re
import random
import torch
import datetime
import time

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertForMultipleChoice
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [3]:
# data read
train = pd.read_csv('my_korean_hate_speech\\data\\korean-hate-speech\\labeled\\new_train.tsv', sep='\t')
test = pd.read_csv('my_korean_hate_speech\\data\\korean-hate-speech\\labeled\\new_test.tsv', sep='\t')
print(train.shape, test.shape)
train.head()

(6316, 4) (1580, 4)


,comments,contain_gender_bias,bias,hate
0,세계급 챙녀 ㅋㅋㅋㅋ,True,gender,hate
1,정준이 연예인이냐? 월세 못낸다고 징징대고 십년안에 연기한 작품이 잇기는 함? 쩌리...,False,others,offensive
2,난민같음 ㅋㅋㅋㅋ,False,others,offensive
3,항상 엄마 밥 먼저!!!아가는 그 후에 먹이세요!,False,none,none
4,장나라는 예쁜데 뭔가좀 않끌 리고 이상해보임,False,none,offensive


In [4]:
# 리뷰 문장 추출
sentences = train['comments']
print(sentences[:5], len(sentences))

# 라벨 추출
label_matching = {'none':0, 'offensive':1, 'hate':2}
#label_matching = {'none':0, 'offensive':1, 'hate':1}
labels = []
for item in train['hate'].values:
    labels.append(label_matching[item])
print(labels[:5], len(labels))

0                                          세계급 챙녀 ㅋㅋㅋㅋ
1    정준이 연예인이냐? 월세 못낸다고 징징대고 십년안에 연기한 작품이 잇기는 함? 쩌리...
2                                            난민같음 ㅋㅋㅋㅋ
3                          항상 엄마 밥 먼저!!!아가는 그 후에 먹이세요!
4                             장나라는 예쁜데 뭔가좀 않끌 리고 이상해보임
Name: comments, dtype: object 6316
[2, 1, 1, 0, 1] 6316


In [5]:
# padding을 위해 최대 길이 추출
max_len = 0
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

for sentence in sentences:
    remove_except_letters = re.sub('[\W_]+', '', sentence)
    remove_except_letters = re.sub("^\d+\s|\s\d+\s|\s\d+$", '', remove_except_letters)
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  95


In [6]:
input_ids = []
attention_masks = []

for sentence in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sentence,
                        add_special_tokens = True,
                        max_length = max_len+1,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                        truncation=True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
for i in range(3):
    print('Original: ', sentences[i])
    print('Token IDs:', input_ids[i])

C:\Users\SHIN-DESKTOP\anaconda3\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:2016: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  세계급 챙녀 ㅋㅋㅋㅋ
Token IDs: tensor([  101, 32613, 37568,  9743, 50814,   100,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0])
Original:  정준이 연예인이냐? 월세 못낸다고 징징대고 십년안에 연기한 작품이 잇기는 함? 쩌리중에 쩌리지
Token IDs: tensor([   101,   9670,  54867,  10739,   9568,  96279,  56789, 118728,    136,
          9613,  24982,   9290,  75884,  85634,   9713, 11

In [7]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

5,684 training samples
  632 validation samples


In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [9]:
# setting the device
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1070


In [10]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False
)

model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr = 2e-6, eps = 1e-8)
epochs = 10

# 총 훈련 스텝 : 배치반복 횟수 * epochs
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [12]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# f1 score
def flat_f1_score(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    #val = f1_score(pred_flat, labels_flat)
    # multi-label case
    val = f1_score(pred_flat, labels_flat, average='macro')

    return val


# 시간 표시 함수
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [13]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()

for epoch_i in range(epochs):    
    # ========================================
    #               Training
    # ========================================    
    print('\n======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()
        
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        # As we unpack the batch, we'll also copy each tensor to the GPU using the `to` method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        
        # Forward
        # outputs cantain (loss, logits)
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("\nRunning Validation...")

    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_f1 = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader. 
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            (loss, logits) = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            labels=b_labels)
                    
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        total_eval_f1 += flat_f1_score(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    print("  F1 score: {0:.2f}".format(avg_val_f1))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Valid. F1.': avg_val_f1,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("\nTraining complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 10 ========
Training...

  Average training loss: 1.04
  Training epcoh took: 0:01:52

Running Validation...
  Accuracy: 0.49
  F1 score: 0.36
  Validation Loss: 0.99
  Validation took: 0:00:03

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.96
  Training epcoh took: 0:01:53

Running Validation...
  Accuracy: 0.55
  F1 score: 0.49
  Validation Loss: 0.92
  Validation took: 0:00:03

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.90
  Training epcoh took: 0:01:52

Running Validation...
  Accuracy: 0.57
  F1 score: 0.52
  Validation Loss: 0.90
  Validation took: 0:00:03

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.87
  Training epcoh took: 0:01:50

Running Validation...
  Accuracy: 0.56
  F1 score: 0.49
  Validation Loss: 0.92
  Validation took: 0:00:03

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.85
  Training epcoh took: 0:01:49

Running Validation...
  Accuracy: 0.5

In [14]:
pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Valid. F1.,Training Time,Validation Time
epoch,,,,,,
1,1.04,0.99,0.49,0.36,0:01:52,0:00:03
2,0.96,0.92,0.55,0.49,0:01:53,0:00:03
3,0.90,0.90,0.57,0.52,0:01:52,0:00:03
4,0.87,0.92,0.56,0.49,0:01:50,0:00:03
5,0.85,0.88,0.56,0.51,0:01:49,0:00:03
6,0.83,0.92,0.58,0.52,0:01:49,0:00:03
7,0.80,0.89,0.57,0.52,0:01:52,0:00:03
8,0.79,0.90,0.57,0.52,0:01:52,0:00:03
9,0.77,0.91,0.58,0.51,0:01:50,0:00:03


In [15]:
# Create sentence and label lists
test = test.dropna()
sentences = test['comments']
labels = []
for item in test['hate'].values:
    labels.append(label_matching[item])

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len+1,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt'     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\SHIN-DESKTOP\anaconda3\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:2016: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
# start test
t0 = time.time()

# 평가모드로 변경
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
eval_f1 = 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(prediction_dataloader):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(prediction_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    tmp_eval_f1 = flat_f1_score(logits, label_ids)
    eval_f1 += tmp_eval_f1
    nb_eval_steps += 1
    
    predictions.append(logits)
    true_labels.append(label_ids)

print("\nAccuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("\nF1 score: {0:.2f}".format(eval_f1/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.58

F1 score: 0.53
Test took: 0:00:07
